# Process IMERG data

combine IMERG files into a single dataset, swapping dimensions so it is (time,lat,lon), restrict region, remove extra variables, write out reduced file

In [11]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc, colors
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
from scipy.interpolate import interp2d, RectBivariateSpline
from datetime import datetime, timedelta
import pandas as pd
import cftime
import netCDF4 as nc
import glob
import os

In [17]:
def get_IMERG_path(datetime_in):
    #IMERG_path_FMT = '/huracan/tank4/cornell/ORCESTRA/imerg/late_run_V07/3B-HHR-L.MS.MRG.3IMERG.{year:04d}{month:02d}{day:02d}-S{hour0:02d}{minute0:02d}{second0:02d}-E{hour1:02d}{minute1:02d}{second1:02d}.{minday:04d}.V07B.RT-H5'
    IMERG_path_FMT = '/huracan/tank4/cornell/ORCESTRA/imerg/final_run_v07_TropAtl_AugSept/3B-HHR.MS.MRG.3IMERG.{year:04d}{month:02d}{day:02d}-S{hour0:02d}{minute0:02d}{second0:02d}-E{hour1:02d}{minute1:02d}{second1:02d}.{minday:04d}.HDF5.nc4'

    datetime_0 = datetime_in
    datetime_1 = datetime_in + timedelta(seconds=1799)

    year_0, month_0, day_0, hour_0, minute_0, second_0 = datetime_0.year, datetime_0.month, datetime_0.day, datetime_0.hour, datetime_0.minute, datetime_0.second
    year_1, month_1, day_1, hour_1, minute_1, second_1 = datetime_1.year, datetime_1.month, datetime_1.day, datetime_1.hour, datetime_1.minute, datetime_1.second

    minday = int(np.rint((datetime_0 - datetime(year_0,month_0,day_0,0,0,0)).seconds / 60.))

    IMERG_path = IMERG_path_FMT.format(year=year_0,month=month_0,day=day_0,hour0=hour_0,minute0=minute_0,second0=second_0,hour1=hour_1,minute1=minute_1,second1=second_1,minday=minday)
    
    return IMERG_path

In [21]:
# Read IMERG data using xarray
#IMERG = xr.open_dataset(get_IMERG_path(datetime(2024,8,16,0,0,0)),engine='h5netcdf',group='Grid')
#IMERG = xr.open_dataset(get_IMERG_path(datetime(2024,8,16,0,0,0)))
IMERG = xr.open_dataset('/huracan/tank4/cornell/ORCESTRA/imerg/final_run_V07_TropAtl_AugSep/3B-HHR.MS.MRG.3IMERG.20240930-S233000-E235959.1410.V07B.HDF5.nc4')

# Swap lat & lon dimensions, so it is (time,lat,lon)
IMERG = IMERG.transpose('time','lat','lon',...)

# Restrict the region
#IMERG = IMERG.sel(lat=slice(-10,50),lon=slice(-90,60))

In [22]:
IMERG

<xarray.Dataset> Size: 2MB
Dimensions:        (lat: 601, latv: 2, time: 1, lon: 1000, nv: 2, lonv: 2)
Coordinates:
  * lat            (lat) float32 2kB -20.05 -19.95 -19.85 ... 39.75 39.85 39.95
  * latv           (latv) int32 8B 0 1
  * lon            (lon) float32 4kB -79.95 -79.85 -79.75 ... 19.75 19.85 19.95
  * lonv           (lonv) int32 8B 0 1
  * nv             (nv) int32 8B 0 1
  * time           (time) object 8B 2024-09-30 23:30:00
Data variables:
    lat_bnds       (lat, latv) float32 5kB ...
    precipitation  (time, lat, lon) float32 2MB ...
    time_bnds      (time, nv) object 16B ...
    lon_bnds       (lon, lonv) float32 8kB ...
Attributes:
    FileHeader:                      DOI=10.5067/GPM/IMERG/3B-HH/07;\nDOIauth...
    FileInfo:                        DataFormatVersion=7e;\nTKCodeBuildVersio...
    history:                         2025-05-13 22:50:45 GMT hyrax-1.17.1 htt...
    history_json:                    [{"$schema":"https:\/\/harmony.earthdata...
    Grid.GridHeader:                 BinMethod=ARITHMETIC_MEAN;\nRegistration...
    Grid.fullnamepath:               /Grid
    DODS_EXTRA.Unlimited_Dimension:  time

In [ ]:
plt.contourf(IMERG.lon,IMERG.lat,IMERG.precipitation[0,:,:])
cbar = plt.colorbar()

In [ ]:
# Load IMERG data
file_paths = glob.glob("/huracan/tank1/work/ORCESTRA/imerg/late_run_V07/3B-HHR-L.MS.MRG.3IMERG*")

#Open the files and combine them into a single dataset
IMERG_ds = xr.open_mfdataset(file_paths,engine='h5netcdf',group='Grid',combine='by_coords')

# Swap lat & lon dimensions, so it is (time,lat,lon)
IMERG_ds = IMERG_ds.transpose('time','lat','lon',...)

# Restrict the region
IMERG_ds = IMERG_ds.sel(lat=slice(-10,50),lon=slice(-90,60))

# Remove extra variables
IMERG_reduced = IMERG_ds.drop_vars(['probabilityLiquidPrecipitation','randomError','precipitationQualityIndex'])

#Write out to new netcdf file
IMERG_reduced.to_netcdf('/huracan/tank1/work/ORCESTRA/imerg/imerg_20240809.nc')


In [12]:
IMERG_reduced

<xarray.Dataset> Size: 11GB
Dimensions:        (time: 2928, nv: 2, lon: 1500, lonv: 2, lat: 600, latv: 2)
Coordinates:
  * time           (time) object 23kB 2024-08-01 00:00:00 ... 2024-09-30 23:3...
  * lon            (lon) float32 6kB -89.95 -89.85 -89.75 ... 59.75 59.85 59.95
  * lat            (lat) float32 2kB -9.95 -9.85 -9.75 ... 49.75 49.85 49.95
Dimensions without coordinates: nv, lonv, latv
Data variables:
    time_bnds      (time, nv) object 47kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    lon_bnds       (time, lon, lonv) float32 35MB dask.array<chunksize=(1, 1500, 2), meta=np.ndarray>
    lat_bnds       (time, lat, latv) float32 14MB dask.array<chunksize=(1, 600, 2), meta=np.ndarray>
    precipitation  (time, lat, lon) float32 11GB dask.array<chunksize=(1, 600, 115), meta=np.ndarray>
Attributes:
    GridHeader:  BinMethod=ARITHMETIC_MEAN;\nRegistration=CENTER;\nLatitudeRe...